<a href="https://colab.research.google.com/github/jihun0423/Baseball-pitcher-predict/blob/main/%EC%9E%84%ED%8F%AC%ED%8A%B8_%26_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 이번 분석을 통해 알아보고 싶은 점

야구에서 투수의 성적을 예측할 때에는, 여러가지 변수가 있다.
타자가 투수의 공을 얼마나 잘 치는가, 투수가 속한 팀의 수비 수준은 어떠한가,
투수가 투구하는 구장 환경은 어떠한가 등등 투수가 공을 던지는 능력과는 무관하다고 볼 수 있는 변수들이 매우 많다.

나는 여기서 수비와 운이라는 변수를 제외한 투수의 성적 예측하고자 한다. 따라서, 최대한 투수의 능력과 관련이 있는 변수들만을 사용해 예측을 할 예정이다.

이를테면, 투수가 던지는 공의 구속, 변화 정도, 스핀 수 등은 투수의 순수 능력이다.
다만, 위의 지표만 이용해서 분석을 할 경우, 구속이 빠른 투수가 예측 성적이 좋게 나올 확률이 높다. 따라서 제구력과 관련된 지표들을 여럿 넣었다. 

BABIP 이론에 따른다면, 타자가 배트에 공을 맞추었을때 나타나는 상황(아웃, 안타 등)은 어느 투수이던 간에 확률이 똑같다고 한다. 인플레이 타구의 결과는 투수와 수비수 모두의 책임이다. 따라서 인플레이 타구에 대해 투수의 책임이 있는 부분만을 다루는 스탯이 필요하며 이것이 바로 이번 예측에 사용할 지표인 xwoba 이다.



# 임포트

In [ ]:
# encoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler


from sklearn.preprocessing import MinMaxScaler

# machine learning models
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from xgboost import XGBRegressor
from sklearn.ensemble import ExtraTreesRegressor

!pip install pycaret
from pycaret.regression import *

# pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

!pip install skorch

from skorch import NeuralNetRegressor
from sklearn.base import BaseEstimator, TransformerMixin

import pandas as pd
import numpy as np

from sklearn.model_selection import GridSearchCV, KFold
from sklearn import set_config

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F

import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.8/105.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requi

In [ ]:
! pip install pandasql   # 처음에만 실행

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=d3a5d650e3185ab76de7c2b2f64ac8eab8d53d206452ab4c53db73f0715f5a70
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql


In [ ]:
! pip install SQLAlchemy==1.4.46   # 처음에만 실행

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.9 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.10
    Uninstalling SQLAlchemy-2.0.10:
      Successfully uninstalled SQLAlchemy-2.0.10


# 데이터 출처

데이터는 BASEBALL-SAVANT라는 평소 매우 애용하고 있는 야구 통계 사이트에서 가져왔다. 자신이 원하는 컬럼만을 선택 할 수 있어, 전처리 과정이 편해진다는 장점이 있다.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd/content/gdrive/MyDrive

/content/gdrive/MyDrive


In [ ]:
df=pd.read_csv("stats (7).csv")             # 모델 학습용 train 데이터. 2016~2022 시즌 투수들의 데이터

In [ ]:
valid=pd.read_csv("stats (8).csv")          # 모델 검증용 valid 데이터. 2023 시즌 투수들의 데이터

* 최종 test 데이터는 train 데이터에서 2020~2022 3년간의 투수 스탯을 이용하여 
2023시즌의 투수 스탯을 짐작한 뒤, 모델을 이용해 예상 성적을 추출할 예정이다.
* valid 데이터에 있는 현재(2023.05.02) 까지의 성적은 이미 실제로 일어난 성적이므로 학습이나 test에는 사용하지 않고, 위에서 예상한 2023 성적과 비교하는 용도로 사용할 예정이다.

# 전처리

In [ ]:
df.isnull().sum()

last_name                   0
 first_name                 0
player_id                   0
year                        0
p_k_percent                 0
p_bb_percent                0
xwoba                       0
z_swing_percent             0
z_swing_miss_percent        0
oz_swing_percent            0
oz_swing_miss_percent       0
oz_contact_percent          0
out_zone_percent            0
iz_contact_percent          0
in_zone_percent             0
edge_percent                0
whiff_percent               0
fastball_avg_speed          0
fastball_avg_spin           0
fastball_avg_break          0
breaking_avg_speed         35
breaking_avg_spin          36
breaking_avg_break         35
offspeed_avg_speed        567
offspeed_avg_spin         567
offspeed_avg_break        567
Unnamed: 26              4198
dtype: int64

결측치가 존재. 커브 같은 브레이킹볼이나, 체인지업류의 오프스피드 피치를 던지지 않는 투수들이 있어 생긴 결측치이다.

이 투수들은 저 구종들을 던지지 않으므로 구속, 회전수, 변화를 전부 0으로 둔 뒤,
브레이킹볼을 던지는지, 오프스피드를 던지는지 여부를 라벨 인코딩을 할 예정이다.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4198 entries, 0 to 4197
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   last_name              4198 non-null   object 
 1    first_name            4198 non-null   object 
 2   player_id              4198 non-null   int64  
 3   year                   4198 non-null   int64  
 4   p_k_percent            4198 non-null   float64
 5   p_bb_percent           4198 non-null   float64
 6   xwoba                  4198 non-null   float64
 7   z_swing_percent        4198 non-null   float64
 8   z_swing_miss_percent   4198 non-null   float64
 9   oz_swing_percent       4198 non-null   float64
 10  oz_swing_miss_percent  4198 non-null   float64
 11  oz_contact_percent     4198 non-null   float64
 12  out_zone_percent       4198 non-null   float64
 13  iz_contact_percent     4198 non-null   float64
 14  in_zone_percent        4198 non-null   float64
 15  edge

# 컬럼 정보


*   last_name, first_name, player_id : 선수 이름, id
*   year : 년도
*   p_k_percent : 투수의 삼진율
*   p_bb_percent : 투수의 볼넷율
*   z_swing_percent : 스트라이크존 안에 던진 공의 스윙 유도율
*   z_swing_miss_percent : 스트라이크존 안에 던진 공의 헛스윙율
*   oz_swing_percent : 스트라이크존 밖에 던진 공의 스윙 유도율
*   oz_swing_miss_percent : 스트라이크존 밖에 던진 공의 헛스윙율
*   oz_contact_percent : 스트라이크존 밖에서 배트에 맞은 비율
*   out_zone_percent : 스트라이크존 밖으로 공을 던진 비율
*   iz_contact_percent : 스트라이크존 안에서 배트에 맞은 비율
*   in_zone_percent : 스트라이크존 안으로 공을 던진 비율
*   edge_percent : 스트라이크존 모서리 (보더라인)으로 공을 던진 비율
*   whiff_percent : 헛스윙 유도율
*   fastball_avg_speed,fastball_avg_spin,fastball_avg_break : 투수가 던진 직구 계열 투구의 평균 구속, 회전수, 변화정도
*   breaking_avg_speed,breaking_avg_spin,breaking_avg_break : 투수가 던진 브레이킹볼 (커브, 슬라이더 등)의 평균 구속, 회전수, 변화정도
*   offspeed_avg_speed,offspeed_avg_spin,offspeed_avg_break : 투수가 던진 오프스피드 피치 (체인지업류)의 평균 구속, 회전수, 변화정도

 
*   xwoba : 구하고자 하는 타겟. 수비와 운에 무관하게 기대되는 투수의 성적. 낮을 수록 성적이 좋음






In [ ]:
df.head(3)

,last_name,first_name,player_id,year,p_k_percent,p_bb_percent,xwoba,z_swing_percent,z_swing_miss_percent,oz_swing_percent,...,fastball_avg_speed,fastball_avg_spin,fastball_avg_break,breaking_avg_speed,breaking_avg_spin,breaking_avg_break,offspeed_avg_speed,offspeed_avg_spin,offspeed_avg_break,Unnamed: 26
0,Perez,Oliver,424144,2020,19.4,8.3,0.274,62.3,10.6,23.8,...,89.9,2114,19.4,76.1,2195.0,10.3,NaN,NaN,NaN,NaN
1,Wainwright,Adam,425794,2020,20.6,5.7,0.324,62.9,15.2,30.2,...,87.6,2253,13.5,73.5,2791.0,22.0,81.8,1683.0,15.3,NaN
2,Greinke,Zack,425844,2020,24.5,3.3,0.299,59.7,17.0,26.3,...,88.1,2327,17.6,77.7,2425.0,12.2,86.2,1668.0,14.0,NaN


## 결측치 제거를 위한 피쳐 생성

In [ ]:
df['throws_breaking'] = 1
df.loc[df['breaking_avg_speed'].isnull()==1,'throws_breaking'] =0             # 브레이킹볼을 던지는지 여부에 따른 인코딩

In [ ]:
valid['throws_breaking'] = 1
valid.loc[valid['breaking_avg_spin'].isnull()==1,'throws_breaking'] =0   

In [ ]:
df['throws_offspeed'] = 1
df.loc[df['offspeed_avg_spin'].isnull()==1,'throws_offspeed'] =0             # 오프스피드볼을 던지는지 여부에 따른 인코딩

In [ ]:
valid['throws_offspeed'] = 1
valid.loc[valid['offspeed_avg_spin'].isnull()==1,'throws_offspeed'] =0 

## 결측치, 이상치 제거

In [ ]:
df = df.fillna(0) # 결측치들을 전부 0으로 변경.

In [ ]:
valid = valid.fillna(0)

In [ ]:
# 위의 df.info()에서, 이상한 점을 하나 발견하였다. 브레이킹볼의 구속과 변화정도의 결측치는 35개인데, 브레이킹볼의 회전수 결측치는 36개이다.
# 기입 오류라고 판단하여 이 투수의 정보는 제거하기로 하였다.

In [ ]:
print(len(df[df['breaking_avg_spin']==0]))
print(len(df[df['breaking_avg_speed']==0]))

df=df[~((df['breaking_avg_spin']==0)&(df['breaking_avg_speed']!=0))]            # 브레이킹볼의 회전수는 결측치 이지만 구속은 결측치가 아닌 투수의 정보는 삭제

36
35


## 불필요한 컬럼 제거

In [ ]:
df.columns

Index(['last_name', ' first_name', 'player_id', 'year', 'p_k_percent',
       'p_bb_percent', 'xwoba', 'z_swing_percent', 'z_swing_miss_percent',
       'oz_swing_percent', 'oz_swing_miss_percent', 'oz_contact_percent',
       'out_zone_percent', 'iz_contact_percent', 'in_zone_percent',
       'edge_percent', 'whiff_percent', 'fastball_avg_speed',
       'fastball_avg_spin', 'fastball_avg_break', 'breaking_avg_speed',
       'breaking_avg_spin', 'breaking_avg_break', 'offspeed_avg_speed',
       'offspeed_avg_spin', 'offspeed_avg_break', 'Unnamed: 26',
       'throws_breaking', 'throws_offspeed'],
      dtype='object')

In [ ]:
valid.columns

Index(['last_name', ' first_name', 'player_id', 'year', 'p_k_percent',
       'p_bb_percent', 'xwoba', 'z_swing_percent', 'z_swing_miss_percent',
       'oz_swing_percent', 'oz_swing_miss_percent', 'oz_contact_percent',
       'out_zone_percent', 'iz_contact_percent', 'in_zone_percent',
       'edge_percent', 'whiff_percent', 'fastball_avg_speed',
       'fastball_avg_spin', 'fastball_avg_break', 'breaking_avg_speed',
       'breaking_avg_spin', 'breaking_avg_break', 'offspeed_avg_speed',
       'offspeed_avg_spin', 'offspeed_avg_break', 'throws_breaking',
       'throws_offspeed'],
      dtype='object')

In [ ]:
df= df.drop(columns=['Unnamed: 26'])     
valid=valid.drop(columns=['Unnamed: 26'])
  # 불필요한 컬럼 제거

# 테스트 데이터 생성

In [ ]:
from pandasql import *
q = """select * from df 
where player_id in (select player_id from df group by player_id having count(*) > 2)  order by player_id,year"""
dat1=sqldf(q,locals())                    # 최근 3년간의 성적을 바탕으로 2023시즌의 성적을 예측할 예정이므로, 3년 이상을 뛴 선수들의 리스트만 불러온다

In [ ]:
dat1.head(6)

,last_name,first_name,player_id,year,p_k_percent,p_bb_percent,xwoba,z_swing_percent,z_swing_miss_percent,oz_swing_percent,...,fastball_avg_spin,fastball_avg_break,breaking_avg_speed,breaking_avg_spin,breaking_avg_break,offspeed_avg_speed,offspeed_avg_spin,offspeed_avg_break,throws_breaking,throws_offspeed
0,Colon,Bartolo,112526,2016,16.2,4.0,0.318,62.4,9.6,30.2,...,2148,17.4,82.5,2282.0,7.9,81.4,1683.0,16.2,1,1
1,Colon,Bartolo,112526,2017,13.7,5.4,0.351,64.9,9.1,28.6,...,2124,20.3,81.5,2334.0,8.0,81.3,1678.0,18.1,1,1
2,Colon,Bartolo,112526,2018,12.9,4.0,0.363,62.6,8.9,29.9,...,2171,18.6,80.5,2388.0,5.7,80.8,1657.0,16.1,1,1
3,Belisle,Matt,279571,2016,17.2,3.8,0.299,76.0,12.9,33.5,...,2073,16.9,84.0,2161.0,9.3,88.1,1728.0,9.1,1,1
4,Belisle,Matt,279571,2017,21.9,8.9,0.282,67.3,16.5,32.2,...,2056,19.2,82.6,2184.0,11.8,85.0,1514.0,13.9,1,1
5,Belisle,Matt,279571,2018,15.4,6.8,0.356,73.5,18.8,35.7,...,2037,16.5,83.4,2164.0,8.8,85.2,1364.0,13.4,1,1


In [ ]:
q = """select * from dat1 
where player_id in (select player_id from valid) order by player_id desc, year desc"""
dat2=sqldf(q,locals())                   # 2023시즌 이전에 3년이상 활약한 선수들 중, 2023년에도 여전히 현역인 선수들만 불러온다.

In [ ]:
dat2

,last_name,first_name,player_id,year,p_k_percent,p_bb_percent,xwoba,z_swing_percent,z_swing_miss_percent,oz_swing_percent,...,fastball_avg_spin,fastball_avg_break,breaking_avg_speed,breaking_avg_spin,breaking_avg_break,offspeed_avg_speed,offspeed_avg_spin,offspeed_avg_break,throws_breaking,throws_offspeed
0,Fleming,Josh,676596,2022,17.2,7.1,0.323,68.0,14.2,33.1,...,1790,15.4,79.5,1846.0,13.6,82.2,1691.0,15.3,1,1
1,Fleming,Josh,676596,2021,14.5,6.9,0.330,69.0,12.7,34.6,...,1813,14.5,78.6,1794.0,8.8,81.8,1603.0,13.8,1,1
2,Fleming,Josh,676596,2020,19.2,5.4,0.281,69.6,12.6,34.4,...,1864,14.8,77.8,1802.0,3.7,81.5,1710.0,13.9,1,1
3,Karinchak,James,675916,2022,38.8,13.1,0.267,56.6,22.2,24.9,...,2349,21.0,82.9,2398.0,9.0,0.0,0.0,0.0,1,0
4,Karinchak,James,675916,2021,33.2,13.6,0.292,66.0,26.3,24.2,...,2309,18.7,83.2,2295.0,8.9,0.0,0.0,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1278,Greinke,Zack,425844,2020,24.5,3.3,0.299,59.7,17.0,26.3,...,2327,17.6,77.7,2425.0,12.2,86.2,1668.0,14.0,1,1
1279,Greinke,Zack,425844,2019,23.1,3.7,0.282,63.7,15.8,33.1,...,2318,16.9,77.4,2481.0,12.3,87.3,1779.0,13.2,1,1
1280,Greinke,Zack,425844,2018,23.7,5.1,0.303,62.5,19.2,31.6,...,2329,17.4,78.7,2514.0,11.6,86.5,1738.0,12.8,1,1
1281,Greinke,Zack,425844,2017,26.8,5.6,0.288,63.3,18.7,32.4,...,2382,18.3,80.4,2594.0,11.4,87.1,1821.0,15.3,1,1


In [ ]:
def predict_2023(df):
  pred_2023 = df.iloc[0,4:]*0.5 + df.iloc[1,4:]*0.3 + df.iloc[2,4:]*0.2

  return pred_2023            # 선수마다 최근일수록 가중치를 더 주어 가중 평균을 구한다.

In [ ]:
test = dat2.groupby('player_id').apply(predict_2023) 

In [ ]:
test.head(3)

,p_k_percent,p_bb_percent,xwoba,z_swing_percent,z_swing_miss_percent,oz_swing_percent,oz_swing_miss_percent,oz_contact_percent,out_zone_percent,iz_contact_percent,...,fastball_avg_spin,fastball_avg_break,breaking_avg_speed,breaking_avg_spin,breaking_avg_break,offspeed_avg_speed,offspeed_avg_spin,offspeed_avg_break,throws_breaking,throws_offspeed
player_id,,,,,,,,,,,,,,,,,,,,,
425844,16.31,4.52,0.3238,63.81,13.07,29.00,35.18,64.43,53.81,86.69,...,2285.1,15.68,76.00,2411.8,13.86,86.41,1633.8,13.71,1.0,1.0
445276,32.10,9.88,0.2507,68.73,25.44,30.18,41.35,57.50,46.30,73.86,...,2501.1,18.65,81.52,2430.7,8.38,0.00,0.0,0.00,1.0,0.0
445926,24.16,7.71,0.3209,62.29,17.04,25.97,29.65,70.10,49.76,82.96,...,2152.3,15.28,79.89,2378.3,11.32,85.17,1853.0,16.66,1.0,1.0


In [ ]:
test.loc[test['throws_breaking']<1,'throws_breaking'] = 0
test.loc[test['throws_breaking']==0,['breaking_avg_speed','breaking_avg_spin','breaking_avg_break']] = 0
test.loc[test['throws_offspeed']<1,'throws_offspeed'] = 0
test.loc[test['throws_offspeed']==0,['offspeed_avg_speed','offspeed_avg_spin','offspeed_avg_break']] = 0            
# 가끔 브레이킹볼이나 오프스피드를 던지다가 안던지거나, 혹은 반대의 경우인 선수들이 있다. 이러한 선수들은 대부분 이 구종들의 구사율이 낮거나 중요하지 않은 경우이다.
# 따라서 던지지 않는 걸로 처리한다.

In [ ]:
test.reset_index(inplace=True)